In [2]:
from math import isnan
import os
import pandas as pd
#!pip install pygeocodio
from geocodio import GeocodioClient
import requests
import sqlalchemy
#!pip install colorama
from colorama import Fore, Style
from inspect import getframeinfo, stack
import smtplib, ssl
from datetime import datetime
from pytz import timezone
from sqlalchemy import create_engine
#!pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

key = "a00cb79ec4f97e6760c49740b8ea7a7be6b77bb"
client = GeocodioClient(key)
print(client)


In [3]:
def myprint(message, is_red="", email_also=""):
    try:
        if email_also != "yes":
            frameinfo = getframeinfo((stack()[1][0]))
            file_and_line_info = Fore.LIGHTBLUE_EX + "  (" + frameinfo.filename.split("/")[-1] + ", line " + str(frameinfo.lineno) + ")" + Style.RESET_ALL
        else:
            file_and_line_info = ""
        if is_red == "red":
            print_red(message + file_and_line_info)
        else:
            print(message + file_and_line_info)
    except:
        print(message + "(there was an error with `myprint`)")
        
def create_address_from(address, city, state, zip):
    return handle_null(address) + ", " + handle_null(city) + " " + handle_null(state) + " " + handle_null(str(zip))

def handle_null(object):
    if pd.isnull(object):
        return ""
    else:
        return str(object)
    
def split_into_parts(a_list, max_items_per_part):
    if len(a_list) < max_items_per_part:
        return [a_list]
    else:
        num_parts = len(a_list) // max_items_per_part
        res = []
        start = 0
        end = max_items_per_part
        for num in range(num_parts):
            res.append(a_list[start:end])

            if num != num_parts - 1:
                start += max_items_per_part
                end += max_items_per_part

        res.append(a_list[end:])
        return res

In [4]:

# geocodes a DataFrame `df`, adding columns for latitude, longitude, accuracy, and accuracy type
# worksite_or_housing is either "worksite" or "housing" and specifies whether to geocode housing or worksite columns
# if `check_previously_geocoded, uses the materialized postgres view `previously_geocoded` to geocode addresses that
# we've already geocoded without using Geocodio so as to save credits
# return the DataFrame with geocoding results columns added

def geocode_table(df, worksite_or_housing, check_previously_geocoded=False):
    myprint(f"Geocoding {worksite_or_housing}...")

    if check_previously_geocoded:
        make_query("REFRESH MATERIALIZED VIEW previously_geocoded")
        
        if worksite_or_housing == "worksite":
            df["address_id"] = df.apply(lambda job: (handle_null(job["street_addr_1_txt"]) + handle_null(job["cty_nm"]) +
                                                     handle_null(job["st_cd"]) + handle_null(job["zip_cd"])).lower(), axis=1)
        else:
            df["address_id"] = df.apply(lambda job: (handle_null(job["HOUSING_ADDRESS_LOCATION"]) + handle_null(job["HOUSING_CITY"]) +
                                                     handle_null(job["HOUSING_STATE"]) + handle_null(job["HOUSING_POSTAL_CODE"])).lower(), axis=1)
        df["previously_geocoded"] = False
        df[f"{worksite_or_housing}_lat"] = None
        df[f"{worksite_or_housing}_long"] = None
        df[f"{worksite_or_housing} accuracy"] = None
        df[f"{worksite_or_housing} accuracy type"] = ""

        errors = 0
        for i, job in df.iterrows():
            # won't work if the full address has certain special characters. should probably fix this but it's rather rare
            try:
                previous_geocode = pd.read_sql(f"""SELECT * FROM previously_geocoded WHERE full_address = '{job["address_id"]}'""", con=engine)
            except:
                previous_geocode = pd.DataFrame()
                print(f"""Failed to query previously_geocoded for address '{job["address_id"]}', the {i + 1}th row.""")
                errors += 1
            if not previous_geocode.empty:
                myprint(f"""'{job["address_id"]}' - the {i + 1}th row - is previously geocoded.""")
                assert len(previous_geocode) == 1
                df.at[i, f"{worksite_or_housing}_lat"] = get_value(previous_geocode, "latitude")
                df.at[i, f"{worksite_or_housing}_long"] = get_value(previous_geocode, "longitude")
                df.at[i, f"{worksite_or_housing} accuracy"] = get_value(previous_geocode, "accuracy")
                df.at[i, f"{worksite_or_housing} accuracy type"] = get_value(previous_geocode, "accuracy_type")
                df.at[i, "previously_geocoded"] = True

        print(f"There were {errors} errors checking for previous geocoding.")
        df = df.drop(columns=["address_id"])

        previously_geocoded = df[df["previously_geocoded"]]
        df = df[~(df["previously_geocoded"])]

        print(f"{len(previously_geocoded)} rows have already been geocoded and {len(df)} rows still need to be geocoded.")

        df = df.drop(columns=["previously_geocoded"])
        previously_geocoded.drop(columns=["previously_geocoded"], inplace=True)

    if not df.empty:

        if worksite_or_housing == "worksite":
            addresses = df.apply(lambda job: create_address_from(job["street_addr_1_txt"], job["cty_nm"], job["st_cd"], job["zip_cd"]), axis=1).tolist()
        elif worksite_or_housing == "housing":
            addresses = df.apply(lambda job: create_address_from(job["HOUSING_ADDRESS_LOCATION"], job["HOUSING_CITY"], job["HOUSING_STATE"], job["HOUSING_POSTAL_CODE"]), axis=1).tolist()
        else:
            print_red_and_email("`worksite_or_housing` parameter in geocode_table function must be either `worksite` or `housing` or `housing addendum`", "Invalid Function Parameter")
            return

        # handles case of more than 10000 addresses - because geocodio api won't batch geocode with more than 10000 addresses at once
        addresses_split = split_into_parts(addresses, 9999)
        geocoding_results = []
        for these_addresses in addresses_split:
            geocoding_results += client.geocode(these_addresses)
        assert len(geocoding_results) == len(addresses)


        latitudes, longitudes, accuracies, accuracy_types, i = [], [], [], [], 0
        for result in geocoding_results:
            try:
                results = result['results'][0]
                accuracies.append(results['accuracy'])
                accuracy_types.append(results['accuracy_type'])
                latitudes.append(results['location']['lat'])
                longitudes.append(results['location']['lng'])
            except:
                accuracies.append(None)
                accuracy_types.append(None)
                latitudes.append(None)
                longitudes.append(None)
            i +=1

        i = len(df.columns)
        df[f"{worksite_or_housing}_lat"] = latitudes
        df[f"{worksite_or_housing}_long"] = longitudes
        df[f"{worksite_or_housing} accuracy"] = accuracies
        df[f"{worksite_or_housing} accuracy type"] = accuracy_types
        myprint(f"Finished geocoding {worksite_or_housing}.")

    if check_previously_geocoded:
        df = df.append(previously_geocoded)

    # # uncomment to save excel file with geocoding results
    # now = datetime.now(tz=timezone('US/Eastern')).strftime("%I.%M%.%S_%p_%B_%d_%Y")
    # df.to_excel(f"geocoded_{now}.xlsx")
    # myprint("Backed up geocoding results")

    return df

# geocodes `df` and returns two dataframes - one with accuratly geocoded rows and one with inaccurate rows
def geocode_and_split_by_accuracy(df, table=""):
    if table == "dol_h2b":
        df = geocode_table(df, "worksite", check_previously_geocoded=True)
    elif table == "housing addendum":
        df = geocode_table(df, "housing", check_previously_geocoded=True)
    elif table == "dol_h2a":
        df = geocode_table(df, "worksite", check_previously_geocoded=True)
        df = geocode_table(df, "housing", check_previously_geocoded=True)
    else:
        df = geocode_table(df, "worksite")
        if "HOUSING_ADDRESS_LOCATION" in df.columns:
            df = geocode_table(df, "housing")
        else:
            print_red_and_email("Not geocoding housing because HOUSING_ADDRESS_LOCATION is not present. This should be fine, and hopefully just means there were only H-2B jobs in today's run, but you may want to check.", "Not geocoding housing today")

    housing_addendum = (table == "housing addendum")
    accurate = df.apply(lambda job: is_accurate(job, housing_addendum=housing_addendum), axis=1)
    accurate_jobs, inaccurate_jobs = df.copy()[accurate], df.copy()[~accurate]
    inaccurate_jobs["fixed"] = False

    myprint(f"There were {len(accurate_jobs)} accurate jobs.\nThere were {len(inaccurate_jobs)} inaccurate jobs.")

    return accurate_jobs, inaccurate_jobs

In [5]:
url = "https://enfxfr.dol.gov/data_catalog/WHD/whd_whisard_20210415.csv.zip"
raw_dol = pd.read_csv(url, 
                      index_col=None,
                      dtype={7:'string'})
raw_dol['findings_start_date'] = pd.to_datetime(raw_dol['findings_start_date'], errors='coerce')
raw_dol['findings_end_date'] = pd.to_datetime(raw_dol['findings_end_date'], errors='coerce')
print(f'raw dataframe has {len(raw_dol.columns)} columns and {len(raw_dol)} rows')

raw dataframe has 110 columns and 313928 rows


In [18]:
list_states= ['TX', 'MS', 'LA', 'KY', 'AL', 'TN']
raw_dol_states = raw_dol[raw_dol["st_cd"].isin(list_states)]
raw_dol_states = raw_dol_states[raw_dol_states["h2a_violtn_cnt"] > 0]
raw_dol_states = raw_dol_states[["street_addr_1_txt", "cty_nm", "st_cd", "zip_cd", "naic_cd", "h2a_violtn_cnt"]]
# 806 rows
practice = raw_dol_states[:10]
practice

                 street_addr_1_txt             cty_nm st_cd   zip_cd naic_cd  \
174     210 Davis Singletary  Road         Waynesboro    MS  39367.0  111334   
2089      9960 Padgett Switch Road          Irvington    AL  36544.0   11142   
2423                1090 Banker Rd  Saint Martinville    LA  70582.0  111930   
2915            4644 Mulberry Pike           Eminence    KY  40019.0  111910   
4147              830 Stewart Road              Wingo    KY  42088.0  111910   
...                            ...                ...   ...      ...     ...   
313236        4024 Money Sunk Road         Yazoo City    MS  39194.0  112511   
313237             5375 MS Hwy 404       Gore Springs    MS  38929.0  111211   
313238             1451 Old Hwy 26         Perkinston    MS  39573.0  111421   
313239             102 Main Street          Itta Bena    MS  38941.0  112511   
313477           1196 Grigsby Lane              Cadiz    KY  42211.0  111910   

        h2a_violtn_cnt  
174           

,street_addr_1_txt,cty_nm,st_cd,zip_cd,naic_cd,h2a_violtn_cnt
174,210 Davis Singletary Road,Waynesboro,MS,39367.0,111334,7
2089,9960 Padgett Switch Road,Irvington,AL,36544.0,11142,2
2423,1090 Banker Rd,Saint Martinville,LA,70582.0,111930,7
2915,4644 Mulberry Pike,Eminence,KY,40019.0,111910,15
4147,830 Stewart Road,Wingo,KY,42088.0,111910,9
4990,8011 Wallonia Road,Cadiz,KY,42211.0,111910,91
6063,12780 Old Palestine Road,Crofton,KY,42217.0,111910,2
6738,929 O'Neil Rd,Newport,TN,37821.0,111219,107
6897,885 FM 1970 North,Timpson,TX,75975.0,115115,506
6898,22602 Fletcher Rd.,Ponchatoula,LA,70454.0,111333,91


In [14]:
geocode_table(practice, "worksite", check_previously_geocoded=False)

Geocoding worksite...  (<ipython-input-4-ab3785178427>, line 8)
Finished geocoding worksite.  (<ipython-input-4-ab3785178427>, line 92)


<ipython-input-4-ab3785178427>:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{worksite_or_housing}_lat"] = latitudes
<ipython-input-4-ab3785178427>:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{worksite_or_housing}_long"] = longitudes
<ipython-input-4-ab3785178427>:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

,street_addr_1_txt,cty_nm,st_cd,zip_cd,naic_cd,h2a_violtn_cnt,worksite_lat,worksite_long,worksite accuracy,worksite accuracy type
174,210 Davis Singletary Road,Waynesboro,MS,39367.0,111334,7,31.595812,-88.654747,1.0,range_interpolation
2089,9960 Padgett Switch Road,Irvington,AL,36544.0,11142,2,30.484272,-88.217554,1.0,rooftop
2423,1090 Banker Rd,Saint Martinville,LA,70582.0,111930,7,30.130344,-91.792026,1.0,rooftop
2915,4644 Mulberry Pike,Eminence,KY,40019.0,111910,15,38.344343,-85.148068,0.8,rooftop
4147,830 Stewart Road,Wingo,KY,42088.0,111910,9,36.595500,-88.647542,1.0,range_interpolation
4990,8011 Wallonia Road,Cadiz,KY,42211.0,111910,91,36.943856,-87.769551,1.0,range_interpolation
6063,12780 Old Palestine Road,Crofton,KY,42217.0,111910,2,36.976458,-87.542592,1.0,range_interpolation
6738,929 O'Neil Rd,Newport,TN,37821.0,111219,107,35.926513,-83.174720,1.0,rooftop
6897,885 FM 1970 North,Timpson,TX,75975.0,115115,506,31.923711,-94.426714,1.0,rooftop
6898,22602 Fletcher Rd.,Ponchatoula,LA,70454.0,111333,91,30.456575,-90.358036,1.0,rooftop


In [ ]:
# questions:
# do we need worksite accuracy and worksite accuracy type
# what should we do with the census data? 
# won't need to merge back on dol if we just keep all of our columns in that we want 